In [24]:
from keras.applications import MobileNet
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D
import numpy as np
from  PIL import Image
from keras.preprocessing import image
from os import listdir
from os.path import isfile, join, isdir
from keras.optimizers import RMSprop
import pandas as pd

In [25]:
model=MobileNet(weights='imagenet',include_top=False,input_shape=(224,224,3))#Loading Model

In [26]:
model.summary()#Summary of model loaded

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [27]:
model.layers[0].input

<tf.Tensor 'input_2:0' shape=(None, 224, 224, 3) dtype=float32>

In [28]:
for layer in model.layers:# Freezing the layers
    layer.trainable=False

In [29]:
model.output

<tf.Tensor 'conv_pw_13_relu_1/Relu6:0' shape=(None, 7, 7, 1024) dtype=float32>

In [30]:
new_model=model.output# adding FC layer
new_model=GlobalAveragePooling2D()(new_model)
new_model = Dense(1024, activation='relu')(new_model)
new_model = Dense(512, activation='relu')(new_model)
new_model = Dense(256, activation='relu')(new_model)
new_model = Dense(128, activation='relu')(new_model)
new_model = Dense(2, activation='sigmoid')(new_model)#value of units=2 as we have 2 classes in label change according to use

In [31]:
Face_model=Model(inputs = model.input , outputs = new_model )#Combining models

In [32]:
Face_model.summary()#Summary of final model

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [33]:
data_path = 'd://faces//training//'#Path where image data is stored
Training_Data,Labels=[],[]
folders=list(filter(lambda x: isdir(join(data_path, x)), listdir(data_path)))

for i in folders:
    onlyfiles = [f for f in listdir(join(data_path, i)) if isfile(join(join(data_path, i), f))]

    # Create arrays for training data and labels
    
    # Open training images in our datapath
    # Create a numpy array for training data
    for i1, files in enumerate(onlyfiles):
        image_path = join(join(data_path, i),onlyfiles[i1])
        images = Image.open(image_path)
        images=images.resize((224,224))
        images=image.img_to_array(images)
        Training_Data.append(np.asarray(images, dtype=np.uint8))
        if i=='pranshul':#Name of 2 person to be identified, both are name of folder in which images are stored
            Labels.append('Pranshul')
        elif i=='tvesha':
            Labels.append('Tvesha')

    # Create a numpy array for both training data and labels
Label_final=pd.DataFrame(data=Labels, columns=['Name'])#
Label_final=pd.get_dummies(Label_final)#Converting categorical data to dummy data
Label_final=Label_final.values#converting pandas to numpy
#print(Label_final)
print("images loaded")

images loaded


In [34]:
Training_Data=np.array(Training_Data)()
Training_Data.shape

(330, 224, 224, 3)

In [35]:
#Optimizer used is RMSprop and loss function=binary_crossentropy
Face_model.compile(loss = 'binary_crossentropy',optimizer = RMSprop(lr = 0.001),metrics = ['accuracy'])#compiling model

In [36]:
Face_model.fit(Training_Data,Label_final,epochs=5)#Training the model

Epoch 1/5
330/330 [==============================] - 49s 148ms/step - loss: 2.3349 - accuracy: 0.6000
Epoch 2/5
330/330 [==============================] - 46s 141ms/step - loss: 0.1497 - accuracy: 0.9394
Epoch 3/5
330/330 [==============================] - 47s 142ms/step - loss: 0.5466 - accuracy: 0.8924
Epoch 4/5
330/330 [==============================] - 47s 141ms/step - loss: 0.0431 - accuracy: 0.9894
Epoch 5/5
330/330 [==============================] - 46s 140ms/step - loss: 0.0054 - accuracy: 0.9985


In [37]:
data_path = 'd://faces//testing//'
Testing_Data,Labels1=[],[]
folders=list(filter(lambda x: isdir(join(data_path, x)), listdir(data_path)))

for i in folders:
    onlyfiles = [f for f in listdir(join(data_path, i)) if isfile(join(join(data_path, i), f))]

    # Create arrays for training data and labels
    
    # Open training images in our datapath
    # Create a numpy array for training data
    for i1, files in enumerate(onlyfiles):
        image_path = join(join(data_path, i),onlyfiles[i1])
        images = Image.open(image_path)
        images=images.resize((224,224))
        images=image.img_to_array(images)
        Testing_Data.append(np.asarray(images, dtype=np.uint8))
        if i=='pranshul':
            Labels1.append('Pranshul')
        elif i=='tvesha':
            Labels1.append('Tvesha')

    # Create a numpy array for both training data and labels
#    Labels = np.asarray(Labels)
Label_test=pd.DataFrame(data=Labels1, columns=['Name'])
Label_test=pd.get_dummies(Label_test)
Label_test=Label_test.values
#print(Label_test)
print("images loaded")

images loaded


In [38]:
Testing_Data=np.array(Testing_Data)
#print(ts)
Testing_Data.shape

(70, 224, 224, 3)

In [54]:
y_pred=Face_model.predict(Testing_Data)

In [55]:
y_pre=[np.where(i==max(i)) for i in y_pred]

In [56]:
l=[np.where(i==max(i)) for i in Label_test]

In [57]:
l[0][0][0]
count=0

In [58]:
for i in range(len(l)): 
    if (l[i][0][0]==y_pre[i][0][0]):
        count+=1

In [59]:
count

57

In [60]:
accuracy=count/len(l)
print('Accuracy of Model:',accuracy*100,"%")

Accuracy of Model: 81.42857142857143 %


In [47]:
Face_model.save('Trasfer_Learning_based_face_recog.pk1')